In [ ]:
from osgeo import gdal
import matplotlib.pyplot as plt
import sklearn
import os
import glob
import numpy as np

In [ ]:
filenames = glob.glob("/content/drive/Shareddrives/My Drive2/colab_agriculture/*.tif")

In [ ]:
filenames

In [ ]:
from xgboost import XGBRegressor
model_xgb = XGBRegressor()
model_xgb.load_model("/content/drive/Shareddrives/My Drive2/LAI_MODEL/LAI_model_no_angle .json")

In [ ]:
timeseries = []
NDVI_DATA = []
times = []
for i in range(len(filenames)):
  dataset = gdal.Open(filenames[i])
  ATMOS =  dataset.GetRasterBand(1).ReadAsArray().astype(float)
  BLUE = dataset.GetRasterBand(2).ReadAsArray().astype(float)
  GREEN = dataset.GetRasterBand(3).ReadAsArray().astype(float)
  RED = dataset.GetRasterBand(4).ReadAsArray().astype(float)
  RE1 = dataset.GetRasterBand(5).ReadAsArray().astype(float)
  RE2 = dataset.GetRasterBand(6).ReadAsArray().astype(float)
  RE3 = dataset.GetRasterBand(7).ReadAsArray().astype(float)
  NIR = dataset.GetRasterBand(8).ReadAsArray().astype(float)
  RE4 = dataset.GetRasterBand(9).ReadAsArray().astype(float)
  ATMOS2 = dataset.GetRasterBand(10).ReadAsArray().astype(float)
  SWIR1 = dataset.GetRasterBand(11).ReadAsArray().astype(float)
  SWIR2 = dataset.GetRasterBand(12).ReadAsArray().astype(float)
  DATA_LAI = np.stack((ATMOS , BLUE , GREEN , RED , RE1 , RE2 , RE3 , NIR , RE4, ATMOS2 , SWIR1 , SWIR2 ) , axis =2)

  DATA_LAI_reshaped = np.reshape(DATA_LAI , (DATA_LAI.shape[0]*DATA_LAI.shape[1],DATA_LAI.shape[2]))
  id_nans = np.isinf(DATA_LAI_reshaped[:,0])
  DATA_LAI_reshaped_nonans = DATA_LAI_reshaped[~id_nans, :] * 10000
  LAI = model_xgb.predict(DATA_LAI_reshaped_nonans)
  LAI = np.reshape(LAI , [LAI.shape[0],1])
  FINAL_LAI = np.zeros([DATA_LAI_reshaped.shape[0],1])
  FINAL_LAI[~id_nans] = LAI
  FINAL_LAI = np.reshape(FINAL_LAI, (dataset.RasterYSize, dataset.RasterXSize))




  NDVI = (NIR - RED)/(NIR + RED)
  NDRE1 = (NIR - RE1)/(NIR + RE1)
  NDRE2 = (NIR - RE2)/(NIR + RE2)
  NDRE3 = (NIR - RE3)/(NIR + RE3)
  MSAVI = (2 * NIR + 1 - np.sqrt((2 * NIR + 1) ** 2 - 8 * (NIR - RED)))/2
  CCCI = ((NIR - RE1)/(NIR + RE1))/((NIR - RED)/(NIR + RED))
  NDWI = (NIR - SWIR2)/(NIR + SWIR2)
  times.append(filenames[i][56 : 64])
  stackedindices = np.stack((NDVI , NDRE1 , NDRE2 , NDRE3 , MSAVI ,CCCI , NDWI) , axis = 2)
  NDVI_DATA.append(NDVI)
  timeseries.append(stackedindices)
  # dataset.append(img)

In [ ]:
DATA_LAI = np.stack((ATMOS , BLUE , GREEN , RED , RE1 , RE2 , RE3 , NIR , RE4, ATMOS2 , SWIR1 , SWIR2 ) , axis =2)
FINAL_LAI = np.zeros([dataset.RasterYSize* dataset.RasterXSize ,])
DATA_LAI_reshaped = np.reshape(DATA_LAI , (DATA_LAI.shape[0]*DATA_LAI.shape[1],DATA_LAI.shape[2]))
id_nans = np.isinf(DATA_LAI_reshaped[:,0])
DATA_LAI_reshaped_nonans = DATA_LAI_reshaped[~id_nans, :]

In [ ]:
DATA_LAI_reshaped_nonans

In [ ]:
DATA_LAI_reshaped_nonans = DATA_LAI_reshaped_nonans * 10000

In [ ]:
LAI = model_xgb.predict(DATA_LAI_reshaped_nonans)
FINAL_LAI[~id_nans] = LAI

In [ ]:
np.max(NDVI)

In [ ]:
id = np.where(FINAL_LAI ==np.max(FINAL_LAI) )
NDVI[id]
np.max(FINAL_LAI)

In [ ]:
FINAL_LAI = np.reshape(FINAL_LAI , (dataset.RasterYSize , dataset.RasterXSize))

In [ ]:
plt.imshow(FINAL_LAI)